# Bitácora Proyeco Bioinformatíca II

### Salvador Alejandro Cuevas Villicaña
### Licenciatura en Ciencias Agrogenómicas
### Escuela Nacional de Estudios Superiores Unidad León
### Universidad Nacional Autónoma de México

Elegí hacer el ensamble del genoma de pseudomonas aeruginosa, una bacteria gram negativa, que es oportunista en plantas y animales causando enfermedades en las vías respiratorias, urinarias, tejidos y generando sepsis. La razón de elegir esta bacteria es por la gran cantidad de información que hay al respecto sobre ella.

La manera en que esta Estrucutrada la bitacora es que esta dividida en cada uno de los puntos solcitados:

#### 0) Descargar el genoma

#### 1) Control de calidad

#### 2) Filtrado de secuencias

#### 3) Calculo de cobertura

#### 4) Mapeo

#### 5) Ensamble del genoma

#### 6) Anotación


Para realizar cada punto me baso en las clases del curso y en algunos artículos, y en cada punto cito si es en el curso y/o en un artículo.

<br>

<br>

<br>

### 0) Descagar el genoma

Descarge el genoma de pseudomonas aeruginosa de la base de datos SRA del NCBI, basandome en lo visto en clase, mediante el comando `fastq-dump --split-3 SRR######`, especificamente la seucencia que elegí tiene el indicador SRR17084770, conectandome desde GAIA el comando que utilice es:

`fastq-dump --split-3 SRR17084738`

La inormación de la secuencia es:

-Size: 282.1Mb

-Instrument: Illumina MiSeq

-Strategy: WGS

-Source: GENOMIC
    
-Selection: RANDOM

-Layout: PAIRED


<br>

### 1) Control de Calidad

El primer análisis de control de calidad se realizó mediante FASTQC, el comando que usé es:

`fastqc SRR*`

Este comando realizo el análisis de ambos archivos obteniendo los resultados del primer control de calidad, donde se vio que la muestra que el SRR17084738_1.fastq muestra mejor calidad que el SRR17084738_2.fastq. Las capturas de pantalla de esto se adjuntan en el PDF, y los html en el repositorio. Pero lo más destacable al respecto es que:

- Tamaño total: 1,024,419 secuencias

- %GC : 65, mas que el porcentaje teoríco

También el FastQC arrojó que la muestra estaba contaminada con el adaptador "nextera transposase sequence", por lo que el siguiente paso fue realizar un filtrado de secuencias

<br>

### 2) Filtrado de secuencias

Antes de realizar el filtrado de secuencias me dio interes por saber en que escala phred se encontrabn mis seceucnias, para no tener porblemas de compatibilidad, y lo realice mediante el comando:

` vsearch --fastq_chars SRR17084738_*`

Evidentemente, antes habia que instalar vsearch, pero una vez esto, se nos confirma que las seceucnias estan en escala phred 33

Para el filtrado de secuencias me apoye en fastp y consultado el manual del mismo opte por poner un filtro de calidad de phred 25 mediante la opción `--qualified_quality_phred` , la razón de elegir un phred de 30 es que elegir un valor más alto eliminaría muchas más secuencias dado la baja calidad que viene en el SRR17084738_2.fastq a comparación del primero, y considerando que el valor de phred 30 da un error de 1 en 1000, considere dejarlo así. Entonces el comando que use queda como:

`fastp -i SRR17084738_1.fastq -I SRR17084738_2.fastq -o filtrado_1.fastq -O filtrado_2.fastq --qualified_quality_phred 30`

Una vez realizado el filtrado se procedió a realizar un analisis mediante FASTQC para compararlos con los orginales. 

`fastqc filtrado_*`

Los archivos FASTQC nuevos nos muestran que si se eliminaron los adaptadores y que el numero total de seucencias que nos quedarón son 766,287

<br>

### 3) Calculo de cobertura

Para el cálculo de cobertura realice los sigueintes comandos:

Cuyo resultado es 766287, es decir el numero total secuencias.

Cuyo resultado es 5300155

Ahora se resta el segundo valor menos el primero para obtener el valor de la cobertura

In [13]:
5300155 - 766287

4533868

Los mismos resultados se obtienen con `filtrado_2.fastq`

<br>

### 4) Mapeo

Para el mapeo me apoye en la herramienta de bowtie2. Entonces una vez teniendo los archivos `filtrado_1.fastq` y `filtrado_2.fastq` en GAIA, primero active el envairoment de bowtie2 con `conda activate bowtie2`. Una vez teniendo esto se construyó el índice de referencia del mapeo con el comando:

`bowtie2-build -f refgenome.fasta REFpsea`

Luego se procedió a realizar el mapeo con el comando:

`bowtie2 -S filtradovsPsea.sam -q --phred33 -p 4 --fr -x  REFpsea -1 filtrado_1.fastq -2 filtrado_2.fastq >& bowtie_psea.log`

Posteriormente por cuestiones de espacio en GAIA baje el archivo de mapeo `filtradovsPsea.sam` a mi computadora para poder eliminar las secuencias que no mapearon mediante el comando:

awk $3!="*" filtradovsPsea.sam >filtradovsREF_reducido.sam

<br>

### 5) Ensamble de genoma

Para el ensamble del genoma me apoye en el software SPAdes instalado en GAIA, el comando usado es:

`spades.py -k 21,33,37,41,55,77 -t 4 -m 8 --pe1-1 filtrado_1.fastq --pe1-2 filtrado_2.fastq -o spades_Psea`

Las opciones del comando usado son -k 21,33,37,41,55,77 que indican los valores de tamaños k-mer que se utilizarán, -t 4 que indica el número de hilos, 4 en este caso, -m que indica el número máximo de memoria RAM que puede usar SPAdes y --pe1-1 y --pe1-2 que indica cual es el archivo 1 y el 2 respectivamente del pareo.

Luego se realizó un análisis de calidad dentro de la carpeta generada `spades_Psea` mediante el comando `quast --split-scaffolds -t 1 scaffolds.fasta`. Se descargaron estos archivos y posteriormente se realizo un analisis de calidad con el genoma de referencia mediante el comando: `quast.py --split-scaffolds -t 1 -r ../refgenome.fasta scaffolds.fasta`. Y se prosiguió a comparar ambos resultados.

<br>

### 6) Anotación

Una vez teniendo los resultados del ensamble procederemos a separar los scaffolds obtenidos en el archivo `scaffolds.fasta`, para poder realizar la anotación de los mismos. Por términos prácticos solo realice la preparación del primer scaffold porque el procedimiento es el mismo para los demás, 80 en este caso.

Primero se convierte el archivo `scaffolds.fasta` a formato tabular mediante el comando:

Luego se calcula la longitud de la última columna de `scaffolds.tab` que es la longitud del scaffolds

Luego se corta esa última columna que nos indica el valor numérico de la longitud del scaffols, mediante el comando:

`cut -f 4 seqs_length.tab > onlylength.list`

Luego con `cat onlylength.list | more` vemos la longitud de los Scaffolds que tenemos. Elegí los dos primeros: es decir los de longitud: 668648 y 521700

Ahora vamos a ver la nomenclatura de estos scaffolds para poder sepáralos del resto. Esto lo hice mediante `head -n 1 seqs_length.tab | cut -f 1`. Dando como resultado que los dos primeros scaffols se llaman:

- NODE_1_length_668648_cov_39.096881


Ahora con el comando `vim` realizo un archivo `.list` donde ingrese el nombre del scaffold correspondiente.:

`vim SCA1.list`


Luego se realizó el corte de dicho scaffold con:

Ahora teniendo nuestro scaffold NODE_1 procedermos a realizar la anotacion estructural y esto es mediante la interfaz web de augustus `https://bioinf.uni-greifswald.de/augustus/`. Como parametros elegi como organismo de referencia a *escherichia coli* Report genes en both strands y Alternative transcripts en middle

Luego de correr augusuts y de comparar las partes graficas del reporte. Extraje una parte del predicted amino acid sequences y del predicted coding sequences de cada scaffold para poder realizar un blast y comprobar resultados. Los genes g401.t1 al g425.t1 fueron los que elegi para realzar la anotación funcional

Los archivos los relice con vim:

`vim aminoacids.faa`

`vim nucleotids.fna`


Primero realicé un blastx con erl archivo de nucleotidos.fna contra la base de datos de non-redundant protein sequences (nr) y los demás datos en default. Dándome de regreso que codifica cada gen, su longitud y porcentaje de identidad. Luego corrí el mismo archivo nucleotidos.fna en Blast2GO donde al compararlo con las bases de datos de interpro me da el mismo tamaño de longitud, pero solo 7 de estos datos lograron completar su BLAST y solo 6 de ellos pudieron mapear y generar anotación. Por lo que combine ambos resultados para poder llegar a mi anotación funcional y por consiguiente a mis resultados y conclusiones.